In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from snowflake.connector.pandas_tools import write_pandas
from sqlalchemy import create_engine
import snowflake.connector

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


In [2]:
import snowflake.connector

In [3]:
## Set options to view all columns
pd.set_option('display.max_columns', None)

In [4]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'FBREF_TEAMSTATS'

In [5]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

In [6]:
cursor = conn.cursor()

In [137]:
cursor.execute('SELECT * FROM TEAMS')
team_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_names = pd.DataFrame(team_rows, columns=column_names)

In [138]:
cursor.execute('SELECT * FROM TEAM_MISC_STATS')
misc_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_misc = pd.DataFrame(misc_rows, columns=column_names)

In [139]:
cursor.execute('SELECT * FROM TEAM_STANDARD_STATS')
standard_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_standard = pd.DataFrame(standard_rows, columns=column_names)

In [140]:
team_logos = \
[["18bb7c10", "https://i.imgur.com/SURo5sj.png"],
 ["8602292d", "https://i.imgur.com/M4mwH1X.png"],
 ["4ba7cbea", "https://i.imgur.com/0F6UIO4.png"],
 ["cd051869", "https://i.imgur.com/MOhGOQ4.png"],
 ["d07537b9", "https://i.imgur.com/EMCwD3X.png"],
 ["cff3d9bb", "https://i.imgur.com/6F7h3UR.png"],
 ["47c64c55", "https://i.imgur.com/LPhzr0K.png"],
 ["d3fd31cc", "https://i.imgur.com/Dgei0uj.png"],
 ["fd962109", "https://i.imgur.com/2zggSrF.png"],
 ["5bfb9659", "https://i.imgur.com/2tipH85.png"],
 ["a2d435b3", "https://i.imgur.com/ldOUO84.png"],
 ["822bd0ba", "https://i.imgur.com/fs7VK9G.png"],
 ["b8fd03ef", "https://i.imgur.com/FF8z7uZ.png"],
 ["19538871", "https://i.imgur.com/z47emBQ.png"],
 ["b2b47a98", "https://i.imgur.com/3kOLokZ.png"],
 ["e4a775cb", "https://i.imgur.com/z4Tl9Zu.png"],
 ["33c895d4", "https://i.imgur.com/35GrgJC.png"],
 ["361ca564", "https://i.imgur.com/fJ7rUWW.png"],
 ["7c21e445", "https://i.imgur.com/9BQHJRl.png"],
 ["8cec06e1", "https://i.imgur.com/6azSF88.png"]
 ]

In [141]:
team_logos = pd.DataFrame(team_logos, columns=['TEAM_FBREF_ID', 'LOGO_URL'])

In [142]:
team_misc = team_misc.merge(team_names, on='TEAM_FBREF_ID', how='left')

In [143]:
team_misc = team_misc.merge(team_standard, on=['TEAM_FBREF_ID', 'SEASON'], how='left')

In [144]:
team_misc = team_misc.merge(team_logos, on=['TEAM_FBREF_ID'], how='left')

In [145]:
team_misc['AERIAL DUELS WON RATIO (%)'] = team_misc['AERIALS_WON']*100/(team_misc['AERIALS_WON'] + 
                                                                team_misc['AERIALS_LOST'])

In [146]:
team_misc['AERIAL DUELS ATTEMPTED PER GAME'] = (team_misc['AERIALS_WON'] + 
                                           team_misc['AERIALS_LOST'])/team_misc['MATCHES_PLAYED']

In [147]:
x_min = (team_misc['AERIAL DUELS WON RATIO (%)'].min() // 5)*5

In [148]:
x_max = (team_misc['AERIAL DUELS WON RATIO (%)'].max()//5)*5 + 5

In [149]:
y_min = (team_misc['AERIAL DUELS ATTEMPTED PER GAME'].min()//5)*5

In [150]:
y_max = (team_misc['AERIAL DUELS ATTEMPTED PER GAME'].max()//5)*5 + 5

In [151]:
(x_min, x_max, y_min, y_max)

(45.0, 60.0, 20.0, 35.0)

In [202]:
def create_aerial_duels_chart(team_misc, x_min, x_max, y_min, y_max):
    fig = go.Figure()
    mean_aerials_won_ratio = team_misc["AERIAL DUELS WON RATIO (%)"].mean()
    mean_aerials_attempted_per_game = team_misc["AERIAL DUELS ATTEMPTED PER GAME"].mean()

    # Add the scatter plot points
    for index, row in team_misc.iterrows():
        fig.add_trace(go.Scatter(
            x=[row["AERIAL DUELS WON RATIO (%)"]],
            y=[row["AERIAL DUELS ATTEMPTED PER GAME"]],
            mode='markers',
            text=row["TEAM_NAME"],
            marker=dict(
                size=1,  # Set a fixed size or normalize as you prefer
                opacity=0
            ),
            hoverinfo='text',
        ))

        # Add team logo as a layout_image
        fig.add_layout_image(
            dict(
                source=row["LOGO_URL"],
                x=row["AERIAL DUELS WON RATIO (%)"],
                y=row["AERIAL DUELS ATTEMPTED PER GAME"],
                xref="x",
                yref="y",
                sizex=5,  # Adjust the size of the image here
                sizey=5,  # Adjust the size of the image here
                xanchor="center",
                yanchor="middle"
            )
        )

    # Update axes and layout as necessary
    # ...
        
    fig.update_xaxes(range=[x_min, x_max], title="AERIAL DUELS WON RATIO (%)")
    fig.update_yaxes(range=[y_min, y_max], title="AERIAL DUELS ATTEMPTED PER GAME")

    fig.add_shape(
        type='line',
        x0=mean_aerials_won_ratio, y0=fig.layout.yaxis.range[0],  # start of the line
        x1=mean_aerials_won_ratio, y1=fig.layout.yaxis.range[1],  # end of the line
        line=dict(color='White', width=3),
        layer='below'
    )

    # Add a horizontal line at the mean aerials attempted per game
    fig.add_shape(
        type='line',
        x0=fig.layout.xaxis.range[0], y0=mean_aerials_attempted_per_game,  # start of the line
        x1=fig.layout.xaxis.range[1], y1=mean_aerials_attempted_per_game,  # end of the line
        line=dict(color='White', width=3),
        layer='below'
    )

    fig.update_layout(
        width=900,
        height=900,
        showlegend=False,
        paper_bgcolor='rgb(70, 70, 70)',
        plot_bgcolor='rgb(70, 70, 70)',
        font=dict(
                family="Roboto, sans-serif",  # Specify the font family
                size=15,                     # Specify the font size
                color="white"                # Specify the font color
            ),
        hoverlabel=dict(
                bgcolor="rgba(20, 20, 20, 0.8)",
                font_family="Roboto, sans-serif"),
        images= [dict(
            source= row["LOGO_URL"],
            xref="x",
            yref="y",
            x= row["AERIAL DUELS WON RATIO (%)"],
            y= row["AERIAL DUELS ATTEMPTED PER GAME"],
            sizex=0.75,  # The size of the image in x axis units
            sizey=0.75,  # The size of the image in y axis units
            sizing="contain",
            layer="above") for index, row in team_misc.iterrows()]
    )

    fig.update_xaxes(
        showline=True,  # Show the axis line
        linewidth=2,  # Width of the axis line
        linecolor='white',  # Color of the axis line
        gridcolor='rgba(0,0,0,0)',  # Set grid line color to transparent
        tickfont=dict(color='white')  # Set the color of the axis ticks (numbers)
    )

    fig.update_yaxes(
        showline=True,
        linewidth=2,
        linecolor='white',
        gridcolor='rgba(0,0,0,0)',
        tickfont=dict(color='white')
    )

    fig.add_annotation(text="Fewer Duels<br>Poor Dueling",
                    xref="paper", yref="paper",
                    x=0, y=0,  # Bottom left corner
                    showarrow=False,
                    font=dict(size=12, color="red", family="Roboto, sans-serif"),
                    align="left")

    fig.add_annotation(text="Lots of Duels<br>Poor Dueling",
                    xref="paper", yref="paper",
                    x=0, y=1,  # Top left corner
                    showarrow=False,
                    font=dict(size=12, color="orange", family="Roboto, sans-serif"),
                    align="left")

    fig.add_annotation(text="Lots of Duels<br>Strong Dueling",
                    xref="paper", yref="paper",
                    x=1, y=1,  # Top right corner
                    showarrow=False,
                    font=dict(size=12, color="green", family="Roboto, sans-serif"),
                    align="right")

    fig.add_annotation(text="Fewer Duels<br>Strong Dueling",
                    xref="paper", yref="paper",
                    x=1, y=0,  # Bottom right corner
                    showarrow=False,
                    font=dict(size=12, color="orange", family="Roboto, sans-serif"),
                    align="right")

    fig.show()

In [203]:
create_aerial_duels_chart(team_misc, x_min, x_max, y_min, y_max)